In [1]:
import pandas as pd
import numpy as np

Loading input into numpy ndarray

In [2]:
def clean_cell(cell: str):
    if cell == "R\n" or "M\n":
        return cell.strip("\n")
    return float(cell)

In [3]:
file = "data/sonar.all-data"
f = open(file)
M = []
for line in f:
    M.append([clean_cell(x) for x in line.split(',')])
M = np.array([np.array(i) for i in M])

Preparing train and val splits

In [4]:
np.random.shuffle(M)
x = M[:, :60].astype(float)
y = M[:, 60]

In [5]:
def normalise_2darray(d2array):
    output_array = []
    for array in d2array:
        x = (array - np.mean(array)) / np.std(array)
        # x[x < 0] *= -1
        # x = (x-np.min(x))/(np.max(x)-np.min(x))
        output_array.append(x)
    return np.asarray(output_array)

x_norm = normalise_2darray(x)
x_norm

array([[-0.78878293, -0.85312577, -0.83563451, ..., -0.84812827,
        -0.85250108, -0.85062702],
       [-0.95337085, -0.88729695, -0.91938999, ..., -0.98357606,
        -0.95676893, -0.96922858],
       [-1.06383584, -1.07817767, -0.98745725, ..., -1.07184058,
        -1.07017292, -1.09518774],
       ...,
       [-0.8787617 , -0.80082221, -0.75391419, ..., -0.94587626,
        -0.94479376, -0.9696911 ],
       [-0.84515252, -0.86290981, -0.85900321, ..., -0.89522809,
        -0.88705973, -0.88741488],
       [-0.7951026 , -0.7925041 , -0.80401172, ..., -0.80289808,
        -0.80252687, -0.81254964]])

In [6]:
def encode_classes(target):
    encoded_target = []
    encoding_dict = {'R': 0, 'M': 1}
    
    for x in target:
        encoded_target.append(encoding_dict.get(x))
            
    return np.asarray(encoded_target), encoding_dict

y_enc, encoding = encode_classes(y)
print(y_enc, '\n', encoding)

[0 0 1 1 0 0 1 1 0 0 0 1 1 1 1 0 0 0 1 0 0 0 0 1 1 1 0 0 1 1 0 0 0 0 1 1 1
 1 0 1 1 1 0 0 1 1 1 0 0 0 1 1 1 1 0 1 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 1
 1 0 0 1 1 0 1 0 1 0 0 1 1 1 1 1 0 0 0 1 1 1 0 0 0 1 0 1 1 1 1 0 1 0 1 0 1
 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 0 1 1 0 0 0 0 1 1 0 0 1 1 0 1 0 1
 0 1 1 0 1 0 1 1 1 1 0 1 1 0 1 1 0 0 1 0 0 1 1 1 0 0 0 1 1 0 1 0 0 0 1 0 1
 0 0 1 1 1 0 0 1 0 1 1 0 0 1 0 1 0 0 1 0 1 0 0] 
 {'R': 0, 'M': 1}


Removing features with low impact on target class

In [7]:
corr_arr = []
for idx, row in enumerate(x_norm):
    arr = np.asarray(list(row) + [y_enc[idx]])
    corr_arr.append(arr)
corr_arr = np.asarray(corr_arr)
corr_map = np.corrcoef(corr_arr, rowvar=False).round(2)
corr_map = corr_map[:, 60]  # keep only final column of the heatmap | correlation to target class
corr_map = corr_map.reshape((61, 1))

to_drop = []
for idx, value in enumerate(corr_map):
    if value > -0.1 and value < 0.1:
        to_drop.append(idx)
to_drop

x_norm = np.delete(x_norm, to_drop, axis=1)
x_norm.shape

(208, 37)

Splitting into train, val and test sets

In [8]:
np.array(np.unique(y_enc, return_counts=True)).T

array([[  0,  97],
       [  1, 111]], dtype=int64)

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test  = train_test_split(x_norm, y_enc, test_size=0.1)
x_train, x_val, y_train, y_val =  train_test_split(x_train, y_train, test_size=0.3)

In [10]:
print(x_train.shape, x_val.shape, x_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

(130, 37) (57, 37) (21, 37)
(130,) (57,) (21,)


In [11]:
np.array(np.unique(y_train, return_counts=True)).T

array([[ 0, 56],
       [ 1, 74]], dtype=int64)

In [12]:
np.array(np.unique(y_val, return_counts=True)).T

array([[ 0, 30],
       [ 1, 27]], dtype=int64)

In [13]:
np.array(np.unique(y_test, return_counts=True)).T

array([[ 0, 11],
       [ 1, 10]], dtype=int64)

Creating model in Keras (tensorflow)

In [14]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.activations import sigmoid, relu, hard_sigmoid, tanh, softmax

input = Input(shape=(x_norm.shape[1], ), name="input")
dense1 = Dense(128, activation=lambda x: relu(x, alpha=0.3))(input,)
dense2 = Dense(128, activation=lambda x: relu(x, alpha=0.3))(dense1)
dense3 = Dense(128, activation=lambda x: relu(x, alpha=0.3))(dense2)
dense4 = Dense(128, activation=lambda x: relu(x, alpha=0.3))(dense3)
dense5 = Dense(64, activation=lambda x: relu(x, alpha=0.3))(dense4)
dense6 = Dense(64, activation=lambda x: relu(x, alpha=0.3))(dense5)
dense7 = Dense(64, activation=lambda x: relu(x, alpha=0.3))(dense6)
dense8 = Dense(64, activation=lambda x: relu(x, alpha=0.3))(dense7)
dense_out = Dense(1, activation=sigmoid, name="dense_out")(dense8)
model = Model(input, dense_out, name='test_model')

c:\Users\vikio\anaconda3\envs\nsiete_test\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (None)/charset_normalizer (3.0.1) doesn't match a supported version!
  warnings.warn(


In [15]:
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from keras.metrics import BinaryAccuracy, AUC

optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.99)
model.compile(loss=BinaryCrossentropy(from_logits=False), optimizer=optimizer, metrics=[BinaryAccuracy(), AUC()])

In [16]:
val_data = (x_val, y_val)
model.fit(x=x_train, y=y_train, validation_data=val_data, batch_size=5, epochs=25, verbose=1)

Epoch 1/25
26/26 [==============================] - 1s 9ms/step - loss: 0.6569 - binary_accuracy: 0.6231 - auc: 0.6389 - val_loss: 0.6023 - val_binary_accuracy: 0.6491 - val_auc: 0.7463
Epoch 2/25
26/26 [==============================] - 0s 3ms/step - loss: 0.5130 - binary_accuracy: 0.7385 - auc: 0.8220 - val_loss: 0.5656 - val_binary_accuracy: 0.7544 - val_auc: 0.8037
Epoch 3/25
26/26 [==============================] - 0s 3ms/step - loss: 0.4493 - binary_accuracy: 0.7846 - auc: 0.8650 - val_loss: 0.4593 - val_binary_accuracy: 0.8421 - val_auc: 0.8623
Epoch 4/25
26/26 [==============================] - 0s 3ms/step - loss: 0.4278 - binary_accuracy: 0.8538 - auc: 0.8866 - val_loss: 0.4788 - val_binary_accuracy: 0.7368 - val_auc: 0.8580
Epoch 5/25
26/26 [==============================] - 0s 3ms/step - loss: 0.3872 - binary_accuracy: 0.8385 - auc: 0.8914 - val_loss: 0.5349 - val_binary_accuracy: 0.7368 - val_auc: 0.8481
Epoch 6/25
26/26 [==============================] - 0s 3ms/step - loss

In [17]:
model.evaluate(x=x_test, y=y_test)

1/1 [==============================] - 0s 7ms/step - loss: 0.7620 - binary_accuracy: 0.8095 - auc: 0.8409


[0.7619737982749939, 0.8095238208770752, 0.8409090042114258]

In [18]:
model.summary()

Model: "test_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 37)]              0         
                                                                 
 dense (Dense)               (None, 128)               4864      
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 128)               16512     
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 64)                4